In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt

In [23]:
df_train = pd.read_csv('train.csv',index_col='Id')
df_test =  pd.read_csv('test.csv',index_col='Id')

In [24]:
no_relevancia = ['index','day', 'NO2', 'O3', 'DEWP', 'station']
df_train.drop(columns= no_relevancia, inplace= True)
df_test.drop(columns= no_relevancia, inplace= True)

In [25]:
df_train["TEMP"] =(df_train["TEMP"]-df_train["TEMP"].min())/(df_train["TEMP"].max()-df_train["TEMP"].min())
df_test["TEMP"] =(df_test["TEMP"]-df_test["TEMP"].min())/(df_test["TEMP"].max()-df_test["TEMP"].min())

In [26]:
def Estandarizar_Direccion(df):
    for idx in df.index:
        valor_cargado = df.loc[idx, "wd"]
        df.loc[idx, "N"] = 1 if "N" in valor_cargado else 0
        df.loc[idx, "E"] = 1 if "E" in valor_cargado else 0

In [27]:
Estandarizar_Direccion(df_train)
Estandarizar_Direccion(df_test)

In [28]:
df_train.drop(columns= ["wd"], inplace= True)
df_test.drop(columns= ["wd"], inplace= True)

In [29]:
df_train["year"] = df_train["year"]-2013
df_test["year"] = df_test["year"]-2013

In [30]:
def Estandarizar_Mes(df):
    grupos = {1:0, 2:1, 3:2, 4:2, 5:2, 6:2, 7:2, 8:3, 9:3, 10:3, 11:3, 12:3}
    for idx in df.index:
        valor_cargado = df.loc[idx, "month"]
        df.loc[idx, "grupo"] = grupos[valor_cargado]

In [31]:
Estandarizar_Mes(df_train)
Estandarizar_Mes(df_test)

In [33]:
X_0 = df_train[df_train["grupo"]==0].drop(columns=["PM2.5", "month", "grupo"])
X_1 = df_train[df_train["grupo"]==1].drop(columns=["PM2.5", "month", "grupo"])
X_2 = df_train[df_train["grupo"]==2].drop(columns=["PM2.5", "month", "grupo"])
X_3 = df_train[df_train["grupo"]==3].drop(columns=["PM2.5", "month", "grupo"])

In [34]:
y_0 = df_train[df_train["grupo"]==0]["PM2.5"]
y_1 = df_train[df_train["grupo"]==1]["PM2.5"]
y_2 = df_train[df_train["grupo"]==2]["PM2.5"]
y_3 = df_train[df_train["grupo"]==3]["PM2.5"]

In [40]:
X_test_0 = df_test[df_test["grupo"]==0].drop(columns=["month", "grupo"])
X_test_1 = df_test[df_test["grupo"]==1].drop(columns=["month", "grupo"])
X_test_2 = df_test[df_test["grupo"]==2].drop(columns=["month", "grupo"])
X_test_3 = df_test[df_test["grupo"]==3].drop(columns=["month", "grupo"])

In [41]:
from sklearn.ensemble import RandomForestRegressor
rfr_0 = RandomForestRegressor(
    n_estimators= 100,
    criterion= "mse",
    min_samples_split= 2,
    min_samples_leaf= 1
)
rfr_1 = RandomForestRegressor(
    n_estimators= 100,
    criterion= "mse",
    min_samples_split= 2,
    min_samples_leaf= 1
)
rfr_2 = RandomForestRegressor(
    n_estimators= 100,
    criterion= "mse",
    min_samples_split= 2,
    min_samples_leaf= 1
)
rfr_3 = RandomForestRegressor(
    n_estimators= 100,
    criterion= "mse",
    min_samples_split= 2,
    min_samples_leaf= 1
)

In [42]:
rfr_0.fit(X_0,y_0)
rfr_1.fit(X_1,y_1)
rfr_2.fit(X_2,y_2)
rfr_3.fit(X_3,y_3)

RandomForestRegressor()

In [43]:
y_pred_0 = rfr_0.predict(X_test_0)
y_pred_1 = rfr_1.predict(X_test_1)
y_pred_2 = rfr_2.predict(X_test_2)
y_pred_3 = rfr_3.predict(X_test_3)

In [45]:
y_pred_0.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [46]:
X_test_0['PM2.5'] =y_pred_0
X_test_1['PM2.5'] =y_pred_1
X_test_2['PM2.5'] =y_pred_2
X_test_3['PM2.5'] =y_pred_3

In [61]:
final_test = pd.concat([X_test_0, X_test_1, X_test_2, X_test_3]) 

In [65]:
final_test.sort_index(ascending=True, inplace= True)

In [66]:
final_test.head()

,year,hour,PM10,SO2,CO,TEMP,PRES,RAIN,WSPM,N,E,PM2.5
Id,,,,,,,,,,,,
0,0,0,4.0,4.0,300.0,0.313214,1023.0,0.0,4.4,1.0,0.0,4.545
2,0,2,7.0,5.0,300.0,0.306688,1023.5,0.0,5.6,1.0,0.0,5.310
12,0,12,6.0,7.0,300.0,0.388254,1027.3,0.0,5.1,1.0,0.0,3.510
15,0,15,15.0,7.0,400.0,0.425775,1025.7,0.0,2.8,1.0,1.0,10.860
17,0,17,23.0,11.0,400.0,0.394780,1026.3,0.0,2.8,1.0,1.0,12.350


In [67]:
final_test['PM2.5'].to_csv('prueba_02', index= True)